# Mining genetic, transcriptomic and imaging data in Parkinson’s disease - 1 

In [ ]:
from matplotlib_venn import venn3
from scipy.stats import norm

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

import subprocess
import math
import sys
import os

## Introduction

#### Required software

- Plink (v1.9b)

#### Data description
PPMI genoptyping data are stored in PLINK file format. They are stored in two genotyping datasets:

- IMMUNO

- NEUROX

IMMUNO dataset targets genetic loci that are known to be associated with major autoimmune and neuroinflammatory diseases. It contains 196,524 genetic variants: 718 indels and 195,806 SNPs. Among these genetic variants, 1920 SNPs replicates PD associated genetic loci.

NeuroX dataset targets ~240,000 exonic genomic variants and ~24,000 custom variants, involved in neurological diseases.

Both datasets provides 3 files:
- BED
- BIM
- FAM

#### PLINK BIM
Plink BIM is a text file with no header, and one line per variant with the following six fields:
- Chromosome code (either an integer, or 'X'/'Y'/'XY'/'MT'; '0' indicates unknown) or name
- Variant identifier
- Position in morgans or centimorgans
- Base-pair coordinate (1-based)
- Allele 1 (usually minor)
- Allele 2 (usually major)

#### PLINK FAM
Plink FAM is a text file with no header, and one line per sample with the following six fields:
- Family ID ('FID')
- Within-family ID ('IID'; cannot be '0')
- Within-family ID of father ('0' if father isn't in dataset)
- Within-family ID of mother ('0' if mother isn't in dataset)
- Sex code ('1' = male, '2' = female, '0' = unknown)
- Phenotype value ('1' = control, '2' = case, '-9'/'0'/non-numeric = missing data if case/control)

#### PLINK BED
Plink BED file is the primary representation of genotype calls at biallelic variants. It must be accompanied by .bim and .fam files. Plink BED file is binary, its human-readable version is the PED file (for further information see [here](https://www.cog-genomics.org/plink/1.9/formats#ped)).

In [ ]:
genotyping_path = "data/genotyping/"
pheno_path = "data/pheno/"
immuno_path = os.path.join(genotyping_path, "IMMUNO")
neurox_path = os.path.join(genotyping_path, "NEUROX")
hapmap_path = os.path.join(genotyping_path, "HapMap3_b37")

## A first glimpse at data

In [ ]:
immuno_bim = pd.read_csv(
    os.path.join(immuno_path, "IMMUNO.bim"), 
    header=None,
    sep="\t"
)
immuno_bim.head()

In [ ]:
neurox_bim = pd.read_csv(
    os.path.join(neurox_path, "NEUROX.bim"),
    header=None,
    sep="\t"
)
neurox_bim.tail()

IMMUNO variant rs9729550 should be mapped at position 1135242 in hg19 release. <br>
If it was mapped on hg18 release it should be mapped at position 1125105. <br>
Therefore, IMMUNO SNPs have been mapped on hg18 assembly!

Moreover, NEUROX SNPs have been mapped on hg19 release.

This means that we need to lift IMMUNO SNPs positions. Let's do it with Plink and UCSC genome browser tools!

First, we need to encode IMMUNO's BIM in a format that the UCSC genome browser can understand: UCSC BED format: 

```chrom start stop name```

In [ ]:
# minimal UCSC BED file requires
#
# CHROM    START    STOP    FEATURE_ID 

tmp_chr = immuno_bim.iloc[:,0]  # chromosomes
tmp_pos = immuno_bim.iloc[:,3]  # SNP positions
tmp_name = immuno_bim.iloc[:,1] # SNP IDs

# append 'chr' in front of chromosome numbers (required by liftOver)
tmp_chr_str = [''.join(["chr", str(c)]) for c in tmp_chr.values]

# replace chr23, chr24, chr25 and chr26 with chrX, chrY, chrX and chrMT
tmp_chr_str2 = [c.replace('23', 'X') for c in tmp_chr_str]
tmp_chr_str3 = [c.replace('24', 'Y') for c in tmp_chr_str2]
tmp_chr_str4 = [c.replace('25', 'X') for c in tmp_chr_str3]
tmp_chr_str5 = [c.replace('26', 'MT') for c in tmp_chr_str4]

# write the resulting UCSC BED file
bed = pd.concat(
    [pd.DataFrame(tmp_chr_str5), tmp_pos, tmp_pos + 1, tmp_name],
    axis=1
)
print(bed.head(n=5))
print(bed.tail(n=5))
bed.to_csv(
    os.path.join(immuno_path, "IMMUNO_tolift.bed"), sep="\t",
    index=False, header=False
)

Let's now use [LiftOver](http://genome.ucsc.edu/cgi-bin/hgLiftOver) tool from UCSC genome browser to lift IMMUNO SNPs positions from hg18 to hg19 genome release. Some SNPs will not be converted (four), but we can ignore it.

In [ ]:
bed_lifted = pd.read_csv(os.path.join(immuno_path, "IMMUNO_lifted.bed"), sep="\t", header=None)

# we keep succesfully mapped SNPs
good_snps = bed_lifted.iloc[:,3]
good_snps.to_csv(
    os.path.join(immuno_path, "good_snps.txt"),
    index=False, header=False
)

# call plink from command line
!plink --bfile {os.path.join(immuno_path, "IMMUNO")} --update-map {os.path.join(immuno_path, "IMMUNO_lifted.bed")} 2 4 --make-bed --extract {os.path.join(immuno_path, "good_snps.txt")} --out {os.path.join(immuno_path, "IMMUNO_hg19")}

In [ ]:
immuno_hg19_bim = pd.read_csv(
    os.path.join(immuno_path, "IMMUNO_hg19.bim"),
    sep="\t",
    header=None
)
immuno_hg19_bim.head()

Now rs9729550 is correctly mapped at position 1135242!

Next we will create a single dataset combining IMMUNO and NEUROX datasets. Plink will help us doing this!

In [ ]:
immuno_fam = pd.read_csv(
    os.path.join(immuno_path, "IMMUNO_hg19.fam"), 
    sep=" ", 
    header=None
)
neurox_fam = pd.read_csv(
    os.path.join(neurox_path, "NEUROX.fam"), 
    sep=" ", 
    header=None
)
# recover subjects IDs
immuno_subj = immuno_fam.iloc[:,1].tolist()
neurox_subj = neurox_fam.iloc[:,1].tolist()

# retrieve subjects with data in both IMMUNO and NEUROX 
common_subj = set(immuno_subj).intersection(set(neurox_subj))

# write common subjects to file
common_subj_fn = "data/genotyping/common_subj.txt"
pd.DataFrame(list(zip(common_subj, common_subj)), columns=["FID", "IID"]).to_csv(
    common_subj_fn,
    header=False,
    index=False,
    sep=" "
)

# call plink on IMMUNO
!plink --bfile {os.path.join(immuno_path, "IMMUNO_hg19")} --keep {common_subj_fn} --make-bed --out {os.path.join(immuno_path, "IMMUNO_common")}
# call plink on NEUROX
!plink --bfile {os.path.join(neurox_path, "NEUROX")} --keep {common_subj_fn} --make-bed --out {os.path.join(neurox_path, "NEUROX_common")}

IMMUNO and NEUROX SNPs were not named using standard variant IDs (e.g. RsIDs). Therefore, ther can be identical SNPs called with different IDs in the two datasets. <br>
To solve this problem we rename each SNP in the two datasets as:<br>
```chrom:pos_A1_A2```

For simplicity we provide the new BIM, BED and FAM files renamed (```IMMUNO_renamed``` and ```NEUROX_renamed``` files).

In [ ]:
immuno_bim = pd.read_csv(
    os.path.join(immuno_path, "IMMUNO_renamed.bim"),
    sep="\t",
    header=None
)
immuno_bim.head()

In [ ]:
# get the names of SNPs only available in IMMUNO and write 
# the SNPs appearing uniquely in IMMUNO to a TXT file
neurox_bim = pd.read_csv(
    os.path.join(neurox_path, "NEUROX_renamed.bim"),
    sep="\t",
    header=None
)
only_immuno_snps_fn = "data/genotyping/onlyIMMUNOsnps.txt"
neurox_snpids_set = set(neurox_bim.iloc[:,1].to_list())
immuno_bim[~immuno_bim[1].isin(neurox_snpids_set)].iloc[:,1].to_csv(
    only_immuno_snps_fn, header=False, index=False
)

# call plink
!plink --bfile {os.path.join(immuno_path, "IMMUNO_renamed")} --extract {only_immuno_snps_fn} --make-bed --out {os.path.join(immuno_path, "IMMUNO_uniquesnps")}

We can now merge the two datasets.

In [ ]:
ppmi_merge_fn = "data/genotyping/PPMI_merge"

!plink --bfile {os.path.join(neurox_path, "NEUROX_renamed")} --bmerge {os.path.join(immuno_path, "IMMUNO_uniquesnps")} --make-bed --out {ppmi_merge_fn}

Plink produces some warnings on SNPs with different positions and same positions with different SNP names. <br>
To make our lives easier, we remove them.

Very easy with Unix shell: <br>
```
cat data/genotyping/PPMI_merge.log | grep 'Warning' | grep 'Variants' | grep -o "'.*'" | cut -d' ' -f1,3 | awk '{print $1"\n"$2;}' | tr -d "'" | sort -u > data/genotyping/merge_rm_snps.txt
```

We can now run again Plink excluding the annoying SNPs!

In [ ]:
!plink --bfile {os.path.join(neurox_path, "NEUROX_renamed")} --bmerge {os.path.join(immuno_path, "IMMUNO_uniquesnps")} --exclude data/genotyping/merge_rm_snps.txt --make-bed --out {ppmi_merge_fn}

Now that our starting dataset is ready, we can move on.

## Quality Control

Quality control (QC) is one of the key steps of GWAS analysis. <br>

QC ensures that potential errors or artifacts (e.g. contaminations, poor quality of DNA samples, etc.) that could provide wrong or biased results are removed from our data.

Let's use Plink to perform QC on PPMI data!

### Individual and SNP missingness

Let's begin by investigating the missingness per individual and per SNPs.

We first remove SNPs with missingess rate > 5%.

In [ ]:
!plink --bfile {ppmi_merge_fn} --geno 0.05 --make-bed --out {"_".join([ppmi_merge_fn, "geno"])}

And remove also those individuals with missingness rate > 5%.

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "geno"])} --mind 0.05 --make-bed --out {"_".join([ppmi_merge_fn, "mind"])}

### Sex discrepancies

Let's continue our QC by performing some simple subject level QC. <br>
We now remove subjects where provided sex information does not match the genotype inferred sex (could be a data quality issue!).

In [ ]:
ppmi_merge_fam = pd.read_csv(
    os.path.join(genotyping_path, "PPMI_merge_mind.fam"),
    sep="\s+",
    header=None
)
ppmi_merge_fam.head()

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "mind"])} --check-sex --out {"_".join([ppmi_merge_fn, "sexcheck"])}

In [ ]:
ppmi_checksex = pd.read_csv(
    ".".join(["_".join([ppmi_merge_fn, "sexcheck"]), "sexcheck"]),
    sep="\s+"
)
ppmi_checksex.head()

In [ ]:
ppmi_checksex[ppmi_checksex.STATUS=="PROBLEM"]

In [ ]:
subj_problem_fn = "data/genotyping/subjs_toremove.txt"
ppmi_checksex[ppmi_checksex.STATUS == "PROBLEM"].iloc[:,0:2].to_csv(
    subj_problem_fn,
    sep=" ",
    index=False
)

!plink --bfile {"_".join([ppmi_merge_fn, "mind"])} --remove {subj_problem_fn} --make-bed --out {"_".join([ppmi_merge_fn, "qc_subjs"])}

For simplicity, we restrict our analysis to autosomal chromosomes.

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "qc_subjs"])} --autosome --make-bed --out {"_".join([ppmi_merge_fn, "autosome"])}

### Minor Allele Frequency (MAF) QC

We continue by removing SNPs with low minor allele frequency. <br>
This depends on dataset size, we used < 1% here.

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "autosome"])} --maf 0.01 --make-bed --out {"_".join([ppmi_merge_fn, "maf"])}

### Hardy-Weinberg equilibrium QC

We also remove those SNPs not following Hardy-Weinberg equilibrium (we used a threshold of $1e^{-6}$ here).

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "maf"])} --hwe 1e-6 --make-bed --out {"_".join([ppmi_merge_fn, "hwe"])}

### Remove ambigous SNPs

Let's now find A/T and C/G SNPs:

In [ ]:
ppmi_merge_bim = pd.read_csv(
    "_".join([ppmi_merge_fn, "hwe.bim"]),
    sep="\t",
    header=None
)
bad1 = ppmi_merge_bim[(ppmi_merge_bim.iloc[:,4] == 'A') & (ppmi_merge_bim.iloc[:,5] == 'T')]
bad2 = ppmi_merge_bim[(ppmi_merge_bim.iloc[:,4] == 'T') & (ppmi_merge_bim.iloc[:,5] == 'A')]
bad3 = ppmi_merge_bim[(ppmi_merge_bim.iloc[:,4] == 'C') & (ppmi_merge_bim.iloc[:,5] == 'G')]
bad4 = ppmi_merge_bim[(ppmi_merge_bim.iloc[:,4] == 'G') & (ppmi_merge_bim.iloc[:,5] == 'C')]

badSnps = pd.concat([bad1, bad2, bad3, bad4])
badSnps_set = set(badSnps.iloc[:,1].values.tolist())

badsnps_fn = "data/genotyping/badsnps.txt"
pd.DataFrame(list(badSnps_set)).to_csv(badsnps_fn, header=False, index=False)

# call plink
!plink --bfile {"_".join([ppmi_merge_fn, "hwe"])} --exclude {badsnps_fn} --make-bed --out {"_".join([ppmi_merge_fn, "snp"])}

### Genetic ancestry QC

Let's now analyse population genetic stratification.<br>
We will merge our dataset with data from the HapMap consortium, who genotyped a lot of people from different populations around the world.

In [ ]:
hp_fn = "data/genotyping/PPMI_HM"
ppmi_merge_bim = pd.read_csv(
    "_".join([ppmi_merge_fn, "snp.bim"]),
    sep="\t",
    header=None
)

# renaming for HM3 was done as described above
hapmap_bim = pd.read_csv(
    os.path.join(hapmap_path, "HM3_b37_renamed.bim"),
    sep="\t",
    header=None
)

# get SNPs in common between the two datasets
A = set(ppmi_merge_bim.iloc[:,1].tolist())
B = set(hapmap_bim.iloc[:,1].tolist())
hp_snps = A.intersection(B)
pd.DataFrame(list(hp_snps)).to_csv(
    "data/genotyping/common_snps.txt",
    index=False,
    header=False
)

# call plink
!plink --bfile {"_".join([ppmi_merge_fn, "snp"])} --bmerge {os.path.join(hapmap_path, "HM3_b37_renamed")} --extract data/genotyping/common_snps.txt --make-bed --out {hp_fn} 

As during the previous merge we got some warnings from Plink on SNPs from different datasets with different names mappoed at the same position. <br>
Let's use again Unix shell to recover them:
```
cat data/genotyping/PPMI_HM.log | grep 'Warning' | grep 'Variants' | grep -o "'.*'" | cut -d' ' -f1,3 | awk '{print $1"\n"$2;}' | tr -d "'" | sort -u > data/genotyping/merge_rm_snps.txt
```

Now we can rerun Plink excluding the annoying SNPs:

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "snp"])} --bmerge {os.path.join(hapmap_path, "HM3_b37_renamed")} --exclude data/genotyping/merge_rm_snps.txt --extract data/genotyping/common_snps.txt --make-bed --out {hp_fn} 

Now we remove SNPs with high missingness in the data (```--geno 0.1```) and low minor allele frequency (```--maf 0.05```).

In [ ]:
!plink --bfile {hp_fn} --geno 0.1 --maf 0.05 --make-bed --out {"_".join([hp_fn, "qc"])}

Now we use PLINK to compute the first few principal components of the relatedness matrix. <br>
This will help us to visualize the population structure.

In [ ]:
!plink --bfile {"_".join([hp_fn, "qc"])} --pca 20 --out {"_".join([hp_fn, "qc"])} 

In [ ]:
hp_pca = pd.read_csv(
    "_".join([hp_fn, "qc.eigenvec"]),
    sep=" ",
    header=None
)
cnames = ['FID','IID']
for i in range(1,21):
    cnames.append("PC"+str(i))
hp_pca.columns = cnames
hp_pca.head()

In [ ]:
def assign_pop(pop):
    if pop.isdigit():
        return "PPMI"
    return pop
hp_pca["FID"] = hp_pca.apply(lambda x : assign_pop(x[0]), axis=1)

# plot population structure
palette = [
    "#000000", 
    "#fbb696", 
    "#913502", 
    "#b48060", 
    "#724a20", 
    "#ffb55f", 
    "#108300", 
    "#006435", 
    "#59dda7", 
    "#508aff", 
    "#53459f", 
    "#cd87ff"
]

# PC1 vs PC2
plt.figure(figsize=(8,8))
sns.scatterplot(data=hp_pca, x="PC1", y="PC2", hue="FID", palette=palette)
plt.title("Population structure", size=16)
plt.xlabel("PC1", size=14)
plt.ylabel("PC2", size=14)

In [ ]:
# PC2 vs PC3
plt.figure(figsize=(8,8))
sns.scatterplot(data=hp_pca, x="PC2", y="PC3", hue="FID", palette=palette)
plt.title("Population structure", size=16)
plt.xlabel("PC2", size=14)
plt.ylabel("PC3", size=14)

We now select only those subjects of European (CEU and TSI) ancestry.

In [ ]:
# we measure the distance from the central european (CEU) and italian (TSI) cluster
hp_ceu = hp_pca[(hp_pca.FID =='CEU') | (hp_pca.FID =='TSI')]
ceu_means = hp_ceu.loc[:,'PC1':'PC5'].apply(np.mean)
ceu_sds   = hp_ceu.loc[:,'PC1':'PC5'].apply(np.std)

#compute z-scores for PPMI subjects
ppmi_pca = hp_pca[hp_pca.FID == "PPMI"]
ppmi_ceu_z = ppmi_pca.loc[:,'PC1':'PC5'].apply(lambda x: (x - ceu_means)/ceu_sds, axis=1)

ppmi_ceu_z.head()

In [ ]:
keep_ppmi = ppmi_ceu_z.apply(lambda x: abs(x) > 5).apply(np.sum,axis=1) == 0

ppmi_color=["b"] * len(keep_ppmi)
idx = [i for i, x in enumerate(keep_ppmi.values) if not x]
for i in idx:
    ppmi_color[i] = "r"

cuse = hp_pca.apply(lambda x: "#000000", axis=1)
plt.figure(figsize=(8,8))
sns.scatterplot(data=hp_pca, x="PC1", y="PC2", hue="FID", palette=palette)
plt.scatter(ppmi_pca.PC1, ppmi_pca.PC2, s=2, c=ppmi_color)
plt.title("Population structure", size=16)
plt.xlabel("PC1", size=14)
plt.ylabel("PC2", size=14)
plt.show()

In [ ]:
ppmi_pca_ceu = ppmi_pca[keep_ppmi]
pd.DataFrame(
    zip(ppmi_pca_ceu.IID.tolist(), ppmi_pca_ceu.IID.tolist())
).to_csv(
    "data/genotyping/ceu_subjs.txt",
    sep=" ",
    index=False,
    header=False
)

# call plink
!plink --bfile {"_".join([ppmi_merge_fn, "snp"])} --keep data/genotyping/ceu_subjs.txt --make-bed --out {"_".join([ppmi_merge_fn, "ceu"])}

We can now recover the subset of samples with complete genotyping, transcriptomic and neuroimaging data for the following analyses. <br>
We also recompute the first 20 PCs of the relatedness matrix which will be used during SNP-phenotype associations.

In [ ]:
!plink --bfile {"_".join([ppmi_merge_fn, "ceu"])} --keep {os.path.join(pheno_path, "rnaseq_subjs.txt")} --make-bed --out {os.path.join(genotyping_path, "PPMI_merge_final")} 

In [ ]:
!plink --bfile {os.path.join(genotyping_path, "PPMI_merge_final")}  --pca 20 --out {os.path.join(genotyping_path, "PPMI_merge_final")} 